## LDA-TM : 
### Extract Topic Model with LdaModel from KCC News and CIS test data

In [1]:
import tensorflow as tf
import keras
#from tensorflow import keras
import pandas as pd
import numpy as np
import gc
import math

In [2]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import gc
import tensorflow as tf
# import keras

# 載入 ckiptagger 自訂詞典
User_Dict = {}
with open("KCC Data/Dict/KccDict2020.txt","r", encoding='utf-8-sig') as UDicts:
    for udic in UDicts:
        udWord = udic.strip().split(" ")
        if len(udWord) > 1:
            User_Dict[udWord[0]] = udWord[1]
        else:
            User_Dict[udWord[0]] = 150    # 未給定權重者一律賦予預設值 150                
dictionary = construct_dictionary(User_Dict)

# 載入模型
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")

# 載入 StopWord
stopword_list = []
with open("KCC Data/Dict/StopWordTest.txt","r", encoding='utf-8-sig') as stopwords:
    for stopword in stopwords:
        stopword_list.append(stopword.strip())
        

C:\Users\User\miniconda3\envs\kcc\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\miniconda3\envs\kcc\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [3]:
zhao = pd.read_csv('zhao_leg.csv',encoding="utf-8-sig") #匯資料
zhao 

,屆,會期,日期,委員,逐字稿
0,9,1,3/16/2016,趙天麟,<U+FEFF>趙委員天麟：主席、各位列席官員、各位同仁。因為今天召委排案審查有關各項選罷法...
1,9,1,3/17/2016,趙天麟,<U+FEFF>趙委員天麟：主席、各位同仁。首先，謝謝主席給我這個機會發言。再來，關於私有老...
2,9,1,3/21/2016,趙天麟,<U+FEFF>趙委員天麟：主席、各位列席官員、各位同仁。主席今天排審政黨法，很多委員都知道...
3,9,1,3/28/2016,趙天麟,<U+FEFF>趙委員天麟：主席、各位列席官員、各位同仁。對主席今天排這個案子，我心情非常激...
4,9,1,3/30/2016,趙天麟,<U+FEFF>趙委員天麟：主席、各位列席官員、各位同仁。我們暫且不論那些大家猜測及推論的黨...
...,...,...,...,...,...
161,9,8,11/13/2019,趙天麟,<U+FEFF>趙委員天麟：（11 時 51 分）主席、各位列席官員、各位同仁。部長，我們全...
162,9,8,11/20/2019,趙天麟,<U+FEFF>趙委員天麟：（9 時 40 分）主席、各位列席官員、各位同仁。臺波雙方沒有邦...
163,9,8,11/22/2019,趙天麟,<U+FEFF>趙委員天麟：（12 時 7 分）主席、各位同仁。所有關心臺灣新式戰鬥機建軍案...
164,9,8,12/13/2019,趙天麟,<U+FEFF>趙委員天麟：（11 時 35 分）主席、各位同仁。將近一年的研究、溝通、審議...


In [6]:
str_str= list(zhao['逐字稿'])
str_str =(','.join('%s' %id for id in str_str)) 
# 開始斷詞
# sentence_segmentation = True,  => To consider delimiters
# segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
# recommend_dictionary = dictionary1, # words in this dictionary are encouraged
# coerce_dictionary = dictionary2, # words in this dictionary are forced
strCut = ws([str_str], sentence_segmentation=True,
                 segment_delimiter_set = {'"',"\r\n","，","「","」",",","？",'\n', "。", ":", "?", "!", ";", "、"},
                 coerce_dictionary = dictionary)

print("------------CKIPTagger Word Segment ws(str,....) + 刪除標點及停用字 ---------------")
stayed_lst= []
for word in strCut[0]:
     # 刪除停用字
    if word not in stopword_list:
        stayed_lst.append(word)
        
print(" / ".join(stayed_lst))


------------CKIPTagger Word Segment ws(str,....) + 刪除標點及停用字 ---------------
趙委員天麟 / 主席 / 列席 / 官員 / 位 / 同仁 / 今天 / 召委 / 排案 / 審查 / 項 / 選罷法 / 案子 / 
 / 
 / 中 / 最為 / 癥結 / 三 / 項 / 三 / 項 / 今天 / 會 / 討論 / 選罷法 / 門檻 / 罷免案 / 門檻 / 
 / 
 / 可否 / 罷免案 / 作 / 宣傳 / 第三 / 原住民 / 民意代表 / 缺額 / 時 / 候選人 / 遞補 / 補選 / 
 / 
 / 問題 / 先 / 門檻 / 部分 / 回顧 / 一下 / 上午 / 已經 / 仔細 / 聆聽 / 委員 / 業務 / 主管 / 就教 / 
 / 
 / 內容 / 回答 / 覺得 / 討論 / 法案 / 不能 / 沒有 / 歷史感 / 最近 / 割 / 闌尾 / 事件 / 
 / 
 / 掀起 / 波濤 / 沒 / 成功 / 引發 / 後面 / 很多 / 政治 / 迴響 / 想 / 請教 / 次長 / 主委 / 
 / 
 / 年 / 門檻 / 提高 / 
 / 
 / 主席 / 請 / 中選會 / 劉 / 主任委員 / 說明 / 
 / 
 / 劉 / 主任委員 / 義周 / 主席 / 委員 / 民國 /  83 /  年 / 門檻 / 提高 / 
 / 
 / 趙委員天麟 / 先 / 請教 / 次長 / 好了 / 知道 / 原因 / 當時 / 歷史 / 背景 / 
 / 
 / 主席 / 請 / 內政部 / 陳 / 次長 / 說明 / 
 / 
 / 陳 / 次長 / 純敬 / 主席 / 委員 / 問題 / 容 / 請 / 業務 / 主管 / 民政 / 司 / 司長 / … / … / 
 / 
 / 
 / 
 / 趙委員天麟 / 先 / 請教 / 主委 / 知道 / 背景 / 
 / 
 / 劉 / 主任委員 / 義周 / 核四 / 立委 / 案子 / 提 / 台北縣 / 案子 / 
 / 
 / 趙委員天麟 / 當時 / 罷免 / 核四 / 立委 / 其實 / 核四 / 議題 / 已經 / 貫穿 / 好幾十 / 年 / 案子 / 國
 / 
 / 民黨 / 執政 / 當時 /

In [8]:
# -*- coding: utf-8 -*-
"""
Input: "KCC Data/CkipNewsCisTest-KccDict2020.txt"
Output: "KCC Data/NewsCisTest-KccDict2020-LDA-TopicNum10.txt"
#CkipNewsCisTest-KccDict2020.txt

@author: Johnson
"""
# When numpy version = 1.19.2 => gensim version should downgrade to 3.8.3
from gensim import corpora, models, utils
from gensim.models import LdaModel, CoherenceModel

###  Read data from txt file, and tokenize words, clean-up text

### Creat dictionary for corpus語料庫

In [9]:
import numpy
stayed_lst_new = [d.split() for d in stayed_lst]
dict = corpora.Dictionary(numpy.asarray(stayed_lst_new))
print("dict = ", dict)
print("------------------------------")

C:\Users\User\miniconda3\envs\kcc\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


dict =  Dictionary(14153 unique tokens: ['趙委員天麟', '主席', '列席', '官員', '位']...)
------------------------------


### Creat corpus need for Topic Modeling

In [10]:
# creat mapping matrix of [word_id, word_frequency] for each word segment
corpus = [dict.doc2bow(text) for text in stayed_lst_new]   
print("corpus[:1] = ", corpus[:1])   
print("------------------------------")

print("print lists of [word:frequency]")
[[print((dict[id], freq)) for id,freq in cp] for cp in corpus[:5]]
print("======================================================")

corpus[:1] =  [[(0, 1)]]
------------------------------
print lists of [word:frequency]
('趙委員天麟', 1)
('主席', 1)
('列席', 1)
('官員', 1)
('位', 1)


### Building the Topic Model from LdaModel

In [11]:
lda_model = models.ldamodel.LdaModel(corpus=corpus, id2word= dict, num_topics=10, passes=20, alpha='auto', chunksize=50, per_word_topics=True)
# print(ldamodel.print_topics())   列出最重要的前若干个主题

output = open("KCC Data/NewsCisTest-KccDict2020-LDA-TopicNum10.txt", 'w', encoding='utf-8-sig')
lstTM = lda_model.print_topics(num_topics=10, num_words=20)
for tupleTM in lstTM:
        str1 = "".join(str(x) for x in tupleTM)
        #print("type(str1) = ", type(str1))
        str1 += "\n"
        print("strTM = ", str1)
        output.write(str1)
output.close()
 
print("------------------------------")
print("Perplexity = ", lda_model.log_perplexity(corpus))

print("------------------------------")
cm = CoherenceModel(model=lda_model, texts=list(stayed_lst_new), dictionary=dict, coherence= 'c_v' )
coher_lda = cm.get_coherence()
print("\n Coherence Score: ", coher_lda)

strTM =  00.244*"位" + 0.004*"28" + 0.000*"64" + 0.000*"1" + 0.000*"2" + 0.000*"日" + 0.000*"5" + 0.000*"7" + 0.000*"立成" + 0.000*"自由派" + 0.000*"公法" + 0.000*"撤換權" + 0.000*"走出" + 0.000*"進出口" + 0.000*"關係人" + 0.000*"利害" + 0.000*"勝出" + 0.000*"淺" + 0.000*"外匯" + 0.000*"緬懷"

strTM =  10.116*"中" + 0.000*"歸於" + 0.000*"緬懷" + 0.000*"綠色" + 0.000*"法律尌" + 0.000*"漫長" + 0.000*"正軌" + 0.000*"撤換權" + 0.000*"自由派" + 0.000*"公法" + 0.000*"外匯" + 0.000*"進出口" + 0.000*"關係人" + 0.000*"利害" + 0.000*"勝出" + 0.000*"贏者" + 0.000*"立成" + 0.000*"解開" + 0.000*"走出" + 0.000*"兩敗俱傷"

strTM =  20.029*"項" + 0.000*"9" + 0.000*"2" + 0.000*"撤換權" + 0.000*"綠色" + 0.000*"緬懷" + 0.000*"歸於" + 0.000*"走出來" + 0.000*"自由派" + 0.000*"公法" + 0.000*"外匯" + 0.000*"進出口" + 0.000*"關係人" + 0.000*"利害" + 0.000*"勝出" + 0.000*"贏者" + 0.000*"立成" + 0.000*"揭諸" + 0.000*"兩敗俱傷" + 0.000*"法律尌"

strTM =  30.108*"討論" + 0.000*"緬懷" + 0.000*"綠色" + 0.000*"法律尌" + 0.000*"漫長" + 0.000*"正軌" + 0.000*"自由派" + 0.000*"公法" + 0.000*"外匯" + 0.000*"撤換權" + 0.000*"進出口" + 0.000*"關係人" + 0.000*"利害" + 0

滿多重疊的，可能是因為篇數太少要分10個主題本來就困難。
貿易strTM =  00.244*"位" + 0.004*"28" + 0.000*"64" + 0.000*"1" + 0.000*"2" + 0.000*"日" + 0.000*"5" + 0.000*"7" + 0.000*"立成" + 0.000*"自由派" + 0.000*"公法" + 0.000*"撤換權" + 0.000*"走出" + 0.000*"進出口" + 0.000*"關係人" + 0.000*"利害" + 0.000*"勝出" + 0.000*"淺" + 0.000*"外匯" + 0.000*"緬懷"

法律strTM =  10.116*"中" + 0.000*"歸於" + 0.000*"緬懷" + 0.000*"綠色" + 0.000*"法律尌" + 0.000*"漫長" + 0.000*"正軌" + 0.000*"撤換權" + 0.000*"自由派" + 0.000*"公法" + 0.000*"外匯" + 0.000*"進出口" + 0.000*"關係人" + 0.000*"利害" + 0.000*"勝出" + 0.000*"贏者" + 0.000*"立成" + 0.000*"解開" + 0.000*"走出" + 0.000*"兩敗俱傷"

法律strTM =  20.029*"項" + 0.000*"9" + 0.000*"2" + 0.000*"撤換權" + 0.000*"綠色" + 0.000*"緬懷" + 0.000*"歸於" + 0.000*"走出來" + 0.000*"自由派" + 0.000*"公法" + 0.000*"外匯" + 0.000*"進出口" + 0.000*"關係人" + 0.000*"利害" + 0.000*"勝出" + 0.000*"贏者" + 0.000*"立成" + 0.000*"揭諸" + 0.000*"兩敗俱傷" + 0.000*"法律尌"

strTM =  30.108*"討論" + 0.000*"緬懷" + 0.000*"綠色" + 0.000*"法律尌" + 0.000*"漫長" + 0.000*"正軌" + 0.000*"自由派" + 0.000*"公法" + 0.000*"外匯" + 0.000*"撤換權" + 0.000*"進出口" + 0.000*"關係人" + 0.000*"利害" + 0.000*"勝出" + 0.000*"贏者" + 0.000*"立成" + 0.000*"解開" + 0.000*"歸於" + 0.000*"走出" + 0.000*"兩敗俱傷"

民主strTM =  40.305*"趙委員天麟" + 0.107*"年" + 0.091*"說" + 0.085*"認為" + 0.081*"件" + 0.025*"先" + 0.000*"協定" + 0.000*"主計" + 0.000*"業界" + 0.000*"縣" + 0.000*"布拉格" + 0.000*"勢力" + 0.000*"駐" + 0.000*"抓走" + 0.000*"政治獻金" + 0.000*"行程" + 0.000*"捐贈" + 0.000*"黨團" + 0.000*"消防" + 0.000*"麵包"

strTM =  50.132*"今天" + 0.000*"撤換權" + 0.000*"綠色" + 0.000*"法律尌" + 0.000*"兩敗俱傷" + 0.000*"歸於" + 0.000*"正軌" + 0.000*"自由派" + 0.000*"公法" + 0.000*"外匯" + 0.000*"進出口" + 0.000*"關係人" + 0.000*"利害" + 0.000*"勝出" + 0.000*"贏者" + 0.000*"立成" + 0.000*"解開" + 0.000*"溫和" + 0.000*"緬懷" + 0.000*"走出"

民主strTM =  60.181*"主席" + 0.154*"次" + 0.110*"問題" + 0.061*"2" + 0.042*"1" + 0.034*"3" + 0.017*"5" + 0.011*"6" + 0.003*"季" + 0.000*"縣" + 0.000*"懇請" + 0.000*"主計" + 0.000*"名稱" + 0.000*"勢力" + 0.000*"代表處" + 0.000*"捐款" + 0.000*"政治獻金" + 0.000*"商人" + 0.000*"澳元" + 0.000*"臺幣"

strTM =  70.554*"會" + 0.000*"懇請" + 0.000*"Taiwan" + 0.000*"布拉格" + 0.000*"China" + 0.000*"到位" + 0.000*"駐" + 0.000*"勢力" + 0.000*"模擬" + 0.000*"捐款" + 0.000*"縣" + 0.000*"干預" + 0.000*"促成" + 0.000*"政治獻金" + 0.000*"代表處" + 0.000*"名稱" + 0.000*"商人" + 0.000*"臺幣" + 0.000*"澳元" + 0.000*"政治獻金法"

strTM =  80.221*"部分" + 0.000*"懇請" + 0.000*"Taiwan" + 0.000*"China" + 0.000*"布拉格" + 0.000*"到位" + 0.000*"駐" + 0.000*"勢力" + 0.000*"模擬" + 0.000*"縣" + 0.000*"捐款" + 0.000*"名稱" + 0.000*"干預" + 0.000*"update" + 0.000*"促成" + 0.000*"政治獻金" + 0.000*"商人" + 0.000*"澳元" + 0.000*"臺幣" + 0.000*"代表處"

strTM =  90.013*"臺灣" + 0.013*"沒有" + 0.008*"支持" + 0.008*"委員" + 0.008*"分" + 0.008*"謝謝" + 0.007*"時" + 0.007*"中國" + 0.007*"現在" + 0.006*"外交部" + 0.006*"人員" + 0.006*"相關" + 0.006*"請" + 0.006*"民主" + 0.006*"國家" + 0.006*"政府" + 0.005*"次長" + 0.005*"非常" + 0.005*"可能" + 0.005*"覺得"

------------------------------
Perplexity =  -11.683328621772267
Coherence Score:  0.726115798738619